In [6]:
import json
from cobra.io import read_sbml_model
from cobra.flux_analysis import find_blocked_reactions
from cobra.flux_analysis import flux_variability_analysis
from fastcore import Fastcore

In [3]:
model = read_sbml_model('data/Recon2.2.xml')

for r in model.reactions:
    r.lower_bound *= 100
    r.upper_bound *= 100
    
with open('data/Recon2.2_blocked_rxns.json') as fh:
    fva_blocked = json.load(fh)
    fva_blocked = set(fva_blocked)

In [4]:
%time fast_cc_blocked = Fastcore.fast_find_blocked(model)
print(len(fast_cc_blocked))

CPU times: user 44.6 s, sys: 20 ms, total: 44.7 s
Wall time: 44.8 s
1921


In [5]:
%time fva_blocked = find_blocked_reactions(model)
print(len(fva_blocked))
fast_cc_blocked - set(fva_blocked)

CPU times: user 3min 31s, sys: 52 ms, total: 3min 31s
Wall time: 3min 31s
1919


{'EX_HC02200_LPAREN_e_RPAREN_', 'NADKm'}

In [13]:
df = flux_variability_analysis(model, reaction_list=list(fast_cc_blocked - set(fva_blocked)))

maximum    0.0
minimum    0.0
Name: NADKm, dtype: float64

In [12]:
from cobra.test import create_test_model
ecoli = create_test_model('textbook')

ecoli.reactions.ATPM.lower_bound = 0
ecoli.reactions.EX_glc__D_e.lower_bound = -10000
ecoli.reactions.EX_o2_e.lower_bound = -10000

%time fcc_blocked = Fastcore.fast_find_blocked(ecoli)
%time fva_blocked = find_blocked_reactions(ecoli)

CPU times: user 72 ms, sys: 0 ns, total: 72 ms
Wall time: 69.9 ms
CPU times: user 28 ms, sys: 0 ns, total: 28 ms
Wall time: 27.5 ms
